In [1]:
import torch
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

def analyze_user_bias(model, data, device='cpu'):
    """
    학습된 모델에서 User Bias(b_u)를 추출하여 분석합니다.
    
    Args:
        model: 학습된 모델 객체
        data: 데이터 로더에서 반환한 딕셔너리 (user_encoder 포함)
    """
    print("Extracting User Bias...")
    
    # 1. 모델에서 b_u 웨이트 꺼내기
    #    model.user_bias는 nn.Embedding(num_users, 1) 입니다.
    #    .weight.data로 텐서에 접근하고, .cpu().numpy()로 넘파이 변환
    if hasattr(model, 'user_bias'):
        # flatten()으로 [Num_Users, 1] -> [Num_Users]로 폅니다.
        bias_weights = model.user_bias.weight.data.cpu().numpy().flatten()
    else:
        print("Error: 모델에 'user_bias' 레이어가 없습니다.")
        return

    # 2. User ID 복원 (인코더 사용)
    user_encoder = data['user_encoder']
    # 0부터 N-1까지의 모든 인덱스를 원래 ID로 변환
    all_indices = range(len(user_encoder.classes_))
    original_user_ids = user_encoder.inverse_transform(all_indices)

    # 3. 데이터프레임 생성
    bias_df = pd.DataFrame({
        'user_idx': all_indices,
        'user_id': original_user_ids,
        'bias_value': bias_weights
    })

    # 4. 통계 및 정렬
    print("-" * 50)
    print(f"Total Users: {len(bias_df)}")
    print(f"Mean Bias: {bias_df['bias_value'].mean():.4f}")
    print(f"Std Bias:  {bias_df['bias_value'].std():.4f}")
    print("-" * 50)

    # 5. 상위 5명 (점수 천사: Generous Users)
    print("\n😇 점수 천사 TOP 5 (Bias가 높은 유저)")
    print(bias_df.nlargest(5, 'bias_value')[['user_id', 'bias_value']])

    # 6. 하위 5명 (점수 악마: Critical Users)
    print("\n😈 점수 악마 TOP 5 (Bias가 낮은 유저)")
    print(bias_df.nsmallest(5, 'bias_value')[['user_id', 'bias_value']])

    # 7. 시각화 (히스토그램)
    plt.figure(figsize=(10, 5))
    sns.histplot(bias_df['bias_value'], bins=50, kde=True, color='skyblue')
    plt.title('Distribution of User Bias ($b_u$)')
    plt.xlabel('Bias Value (Higher = More Generous)')
    plt.ylabel('User Count')
    plt.axvline(0, color='red', linestyle='--', label='Neutral (0.0)')
    plt.legend()
    plt.show()

    return bias_df